In [1]:
from typing import List, DefaultDict
from collections import defaultdict
import re

#### Part 1

    byr (Birth Year)
    iyr (Issue Year)
    eyr (Expiration Year)
    hgt (Height)
    hcl (Hair Color)
    ecl (Eye Color)
    pid (Passport ID)
    cid (Country ID)


In [2]:
test_data = """ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

In [3]:
KEYS = {'ecl', 'pid', 'eyr', 'hcl', 'byr', 'iyr', 'hgt', 'cid'}

Passport = DefaultDict[str,str]
def parse_passport(raw_pass: str, keys= List[str]) -> DefaultDict[str,str]:
    """
        'iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
         hcl:#cfa07d byr:1929'
    """
    passport = defaultdict.fromkeys(keys, None) #not required just explicit
    for chunk in raw_pass.split("\n"):
        for pairs in chunk.split(" "):
            key, val = pairs.split(":")
            passport[key] = val
    return passport

def get_passports(log_str:str, keys=List[str])-> List[Passport]:
    passports = log_str.split("\n\n")
    to_process = []
    for p in passports:
        to_process.append(parse_passport(raw_pass=p, keys=keys))
    return to_process

def validate_passport(to_process: List[Passport], ignore:str = 'cid') -> int:
    return sum(
        all(val for key, val in d.items() if key != ignore )
        for d in to_process
    )           
    

In [4]:
to_process = get_passports(log_str=test_data, keys=KEYS)
assert validate_passport(to_process=to_process) == 2

with open('puzzle_inputs/day04_01.txt', 'r') as file:
    log_str = file.read()
    to_process = get_passports(log_str=log_str, keys=KEYS)
    valid_pass = validate_passport(to_process=to_process)
valid_pass

226

#### Part 2


    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.


In [27]:
def _byr(value:str)-> bool:
     return 1920 <= int(value) <= 2002
    
def _iyr(value:str) -> bool:
    return  2010 <= int(value) <= 2020

def _eyr(value:str) -> bool:
    return  2020 <= int(value) <= 2030

def _hgt(value:str) -> bool:
    if 'in' in value:
        return 59 <= int(value.strip("in")) <= 76
    if 'cm' in value:
        return 150 <= int(value.strip("cm")) <= 193
    return False
    
def _hcl(value:str)-> bool:
    return bool(
    re.match("^#[a-f0-9]{6}", value)
    )

def _ecl(value:str)-> bool:
    colours = {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'} 
    return bool(
    {value.strip()} & colours
    )

def _pid(value:str)-> bool:
    if len(value) != 9:
        return False
    return bool(
    re.match("^[0-9]{9}", value)
    )
    
def validate_fields(passport:Passport, ignore:str = 'cid') -> bool:
    return all(fn(passport[key])
            for key, fn in VAL_FN.items()
            if key != ignore
    )

def validate_keys(passport:Passport , ignore:str = 'cid') -> int:
    return all(val 
               for key, val in passport.items() 
               if key != ignore
    )

VAL_FN = {'byr':_byr,  'ecl':_ecl, 'eyr': _eyr, 'hcl':_hcl, 'hgt': _hgt, 'iyr': _iyr, 'pid':_pid}

In [28]:
def process_passports(to_process: List[Passport], ignore = 'cid') -> int:
    valid_p = 0
    for p in to_process:
        if validate_keys(passport=p) and validate_fields(passport=p):
            valid_p += 1
    return valid_p

In [29]:
TEST = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007"""

In [30]:
to_process = get_passports(log_str=TEST, keys=KEYS)
assert process_passports(to_process=to_process) == 0

In [31]:
TEST = """pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719"""

In [32]:
to_process = get_passports(log_str=TEST, keys=KEYS)
assert process_passports(to_process=to_process) == 4

    byr valid:   2002
    byr invalid: 2003

    hgt valid:   60in
    hgt valid:   190cm
    hgt invalid: 190in
    hgt invalid: 190

    hcl valid:   #123abc
    hcl invalid: #123abz
    hcl invalid: 123abc

    ecl valid:   brn
    ecl invalid: wat

    pid valid:   000000001
    pid invalid: 0123456789

In [35]:
with open('puzzle_inputs/day04_01.txt', 'r') as file:
    log_str = file.read()
    to_process = get_passports(log_str=log_str, keys=KEYS)
    valid_pass = process_passports(to_process=to_process)
valid_pass

160